# Hyperparameter Optimization (Triaxial Model)

In [1]:
import logging
import random
import os
import time
import datetime
import warnings
from itertools import product
from multiprocessing import Pool

import torch
from torch import nn, optim
from torch.optim.lr_scheduler import ExponentialLR
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import h5py
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, KFold

from utils import count_parameters
from models import *

warnings.filterwarnings("ignore")

# Make code deterministic
SEED = 1337
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Config and global variables

In [2]:
# Set file paths
BASE_PATH = os.path.join("/home/source/experiments/")
RESULTS_BASE_PATH = os.path.join(BASE_PATH, "results")
LOG_FILE_PATH = os.path.join(BASE_PATH, "exp04_train.log")
MODEL_BASE_PATH = os.path.join(BASE_PATH, 'exp04_models')
TRAIN_RESULTS_PATH = os.path.join(RESULTS_BASE_PATH, "exp04_train.csv")
TEST_RESULTS_PATH = os.path.join(RESULTS_BASE_PATH, "exp04_test.csv")

# Logging config
logging.basicConfig(level=logging.INFO,
                    filename=LOG_FILE_PATH,
                    format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# Set information about the dataset
HDF5_FILE_PATH = os.path.join(os.sep, 'home', 'data', "ANNOTATED_BEDTIME_TU7.hdf5")
EPOCH_FILE_PATH = os.path.join(os.sep, 'home', 'data', "ACTIGRAPH_EPOCHS_TU7_CLEAN.hdf5")
COLNAMES = ["Time", "X", "Y", "Z", "Annotated Time in Bed"]
SAMPLE_RATE = 100
LABEL_DICT = {False: 0, True: 1}
EXCLUDED_DATASETS = []
EPOCH_LENGTH = "60s"

# Set information about the model, etc.
INPUT_DIM = 3
OUTPUT_DIM = 1

DROPOUT = 0.5 # https://jmlr.org/papers/v15/srivastava14a.html
BATCH_SIZE = 8
CLIP = 1000
MAX_EPOCHS = 256
MIN_EPOCHS = 0
LR_DECAY = .9
REVERSE = False

# Combinations to test
MODELS = [MLP, RNN, LSTM]
HID_DIM = [1,2,4,8,16,32,64]
N_LAYERS = [1,2,4]
INIT_LR = [.7]

# Minimal required loss impprovement
EPSILON = 1e-4

means = torch.Tensor([-0.0933,  0.3356, -0.2729])
stds = torch.Tensor([0.4997, 0.4727, 0.5715])

## Load data

In [3]:
def load_subject(actigraph_file_path, epoch_file_path, subject, target_dict, colnames):
    time_col, x_col, y_col, z_col, target_col = colnames

    data = pd.read_hdf(actigraph_file_path, key=subject)
    data = data[[time_col, target_col]]

    epochs = pd.read_hdf(epoch_file_path, key=subject)

    if min(data[time_col]).strftime('%Y-%d-%m') == min(epochs[time_col]).strftime('%Y-%m-%d'):
        start_time = pd.Timestamp(min(data[time_col]).strftime('%Y-%d-%m %H:%M:%S')) # Apparently d and m are switched in the data.
        data.loc[:,time_col] = pd.date_range(start_time, periods=data.shape[0], freq="10ms")
    else:
        start_time = min(data[time_col])

    end_time = max(data[time_col])

    epochs = epochs.loc[(epochs[time_col] > start_time) & (epochs[time_col] < end_time), :]
    epochs = epochs.set_index(time_col).resample(EPOCH_LENGTH).sum()

    epochs = pd.merge_asof(epochs, data, on=time_col, direction="nearest")
    
    #X = torch.from_numpy(np.array(epochs[y_col].values, dtype=np.float))
    X = torch.from_numpy(np.array(epochs[[x_col, y_col, z_col]].values, dtype=np.float))
    y = torch.from_numpy(epochs[target_col].apply(lambda label: target_dict[label]).values)
    
    return X, y

In [4]:
def load_subject_helper(subject): 
    return load_subject(HDF5_FILE_PATH, EPOCH_FILE_PATH, subject, LABEL_DICT, COLNAMES)


def load_dataset(file_path, epoch_file_path, subjects, label_dict, colnames):
    
    with Pool(200) as p:
        X, y = zip(*list(tqdm(p.imap(load_subject_helper, subjects), total=len(subjects))))

    lengths = [elem.shape[0] for elem in X]

    X, y, lengths = zip(*[(X[ii], y[ii], lengths[ii]) for ii in np.argsort(lengths)[::-1]])

    means, stds = torch.cat(X).mean(axis=0), torch.cat(X).std(axis=0)

    logging.info(f"means = {means}; stds = {stds}")
    print(f"means = {means}; stds = {stds}")

    class_0, class_1 = zip(*[((elem == 0).sum().numpy()/elem.shape[0], (elem == 1).sum().numpy()/elem.shape[0]) for elem in y])
    logging.info(f"Class 0 (awake): {np.mean(class_0):.2f} +/- {np.std(class_0):.2f}; Class 1 (sleep): {np.mean(class_1):.2f} +/- {np.std(class_1):.2f}")
    print(f"Class 0 (awake): {np.mean(class_0):.2f} +/- {np.std(class_0):.2f}; Class 1 (sleep): {np.mean(class_1):.2f} +/- {np.std(class_1):.2f}")

    X, y, lengths = pad_sequence(X, batch_first=True), pad_sequence(y, batch_first=True), torch.Tensor(lengths)

    X = (X - means) / stds
    logging.info("Normalized the input of each channel (see train.py for details)")
    print("Normalized the input of each channel (see train.py for details)")

    return X, y, lengths

# Select device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load available subjects
with h5py.File(HDF5_FILE_PATH) as hdf5_file:
    subjects = [subject for subject in hdf5_file.keys() if subject not in EXCLUDED_DATASETS]
    
# Load the data
X, y, lengths = load_dataset(HDF5_FILE_PATH, EPOCH_FILE_PATH, subjects, LABEL_DICT, COLNAMES)
X, y = X.float(), y.float()
X, y, lengths = X.to(device), y.to(device), lengths.to(device)
assert X.shape[0] == y.shape[0]
print(f"Loaded {X.shape[0]} sequences with input shape [{X.shape[1]} x {X.shape[2]}] and output shape [{y.shape[1]}]\n")

  0%|          | 0/445 [00:00<?, ?it/s]

means = tensor([216.9997, 204.3047, 234.3755], dtype=torch.float64); stds = tensor([472.9218, 612.3429, 486.0117], dtype=torch.float64)
Class 0 (awake): 0.54 +/- 0.11; Class 1 (sleep): 0.46 +/- 0.11
Normalized the input of each channel (see train.py for details)
Loaded 445 sequences with input shape [1665 x 3] and output shape [1665]



## Create result files

In [5]:
with open(TRAIN_RESULTS_PATH, "w") as f:
    f.write("Combination,Fold,Epoch,Train Loss,Validation Loss,Hidden Dimension,Number of Layers,Initial Learning Rate,Model\n")
logging.info(f"Created training result file at {TRAIN_RESULTS_PATH}")

with open(TEST_RESULTS_PATH, "w") as f:
    f.write("Combination,Fold,Loss,Accuracy,Precision,Recall,F1 Score,Hidden Dimension,Number of Layers,Initial Learning Rate,Model,Ellapsed Time\n")
logging.info(f"Created test result file at {TEST_RESULTS_PATH}")

## Train the models

In [6]:
combinations = [(0, INIT_LR[0], 0, GLM)] + list(product(N_LAYERS, INIT_LR, HID_DIM, MODELS))

n_combinations = len(combinations)
for combination, (n_layers, init_lr, hid_dim, model_constr) in enumerate(tqdm(combinations)):

    logging.info(f"Combination {combination}: hid_dim = {hid_dim}; n_layers = {n_layers}; init_lr = {init_lr}; device = {device}")

    # Do 10-fold cross-validation
    kf = KFold(n_splits=10)
    for fold, (train_idx, test_idx) in enumerate(kf.split(np.arange(X.size(0)))):

        # Create validation data
        train_idx, valid_idx = train_test_split(np.arange(train_idx.shape[0]), test_size=0.2)

        # Create model and init weights
        model = model_constr(INPUT_DIM, hid_dim, OUTPUT_DIM, n_layers, dropout=DROPOUT, batch_first=True)
        logging.info('Model initialized with %s trainable parameters' % count_parameters(model))

        # Init loss and optimizer
        optimizer = optim.SGD(model.parameters(), lr=init_lr) # https://arxiv.org/abs/1409.3215
        scheduler = ExponentialLR(optimizer, gamma=LR_DECAY)
        criterion = nn.BCELoss()
        logging.info(f"Start with learning rate = {init_lr} (decay = {LR_DECAY}); batch size = {BATCH_SIZE}.")

        # Create dataloaders
        train_loader = DataLoader(TensorDataset(X[train_idx], y[train_idx], lengths[train_idx]), batch_size=BATCH_SIZE, shuffle=True)
        valid_loader = DataLoader(TensorDataset(X[valid_idx], y[valid_idx], lengths[valid_idx]), batch_size=BATCH_SIZE)
        test_loader = DataLoader(TensorDataset(X[test_idx], y[test_idx], lengths[test_idx]), batch_size=BATCH_SIZE)
        logging.info(f"Use {len(train_idx)} sequences for training, {len(valid_idx)} sequences for validation and {len(test_idx)} sequences for testing.")

        # Set path and init best loss
        best_model_path = os.path.join(MODEL_BASE_PATH, f'{combination:02d}_best_{n_layers}l_{model.name}{hid_dim}_model_fold_{fold}.pt')
        best_valid_loss = float('inf')
        epoch = 0

        overall_start_time = time.time()

        # Evaluate model without any training
        train_loss, _ = evaluate(model, train_loader, criterion)
        valid_loss, _ = evaluate(model, valid_loader, criterion)

        # Save losses to result file
        with open(TRAIN_RESULTS_PATH, "a") as f:
            f.write(f"{combination},{fold},{epoch},{train_loss},{valid_loss},{hid_dim},{n_layers},{init_lr},{model.name}\n")

        for epoch in range(1, MAX_EPOCHS + 1):

            start_time = time.time()

            train_loss = train(model, train_loader, optimizer, criterion, CLIP)
            valid_loss, _ = evaluate(model, valid_loader, criterion)

            time_diff = int(time.time() - start_time)

            scheduler.step()

            if valid_loss + EPSILON < best_valid_loss:
                # Save losses to result file
                with open(TRAIN_RESULTS_PATH, "a") as f:
                    f.write(f"{combination},{fold},{epoch},{train_loss},{valid_loss},{hid_dim},{n_layers},{init_lr},{model.name}\n")

                # Update best validation loss and save model
                best_valid_loss = valid_loss
                logging.info(f"Updated best validation loss to {best_valid_loss}.")
                torch.save(model.state_dict(), best_model_path)
            else:
                logging.info(f"End training after epoch {epoch} as validation loss does not further decrease.")
                logging.info(f"Best model saved at {best_model_path}")
                break

        time_diff = int(time.time() - overall_start_time)

        # Evaluate model on test set
        logging.info(f"Load model from epoch {epoch-1} from {best_model_path}")
        model.load_state_dict(torch.load(best_model_path))

        test_loss, metrics = evaluate(model, test_loader, criterion)
        accuracy, precision, recall, f1_score = metrics

        with open(TEST_RESULTS_PATH, "a") as f:
            f.write(f"{combination},{fold},{test_loss},{accuracy},{precision},{recall},{f1_score},{hid_dim},{n_layers},{init_lr},{model.name},{time_diff}\n")

  0%|          | 0/64 [00:00<?, ?it/s]

RuntimeError: all elements of input should be between 0 and 1

In [15]:
a,b,c = zip(*list(train_loader))

In [21]:
[torch.all((xx >= 0) & (xx <= 1)) for xx in b]

[tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True)]

In [24]:
model.train()
epoch_loss = 0

for i, (X, y, lens) in enumerate(train_loader):
    optimizer.zero_grad()

    y_pred = model(X, lens)
    loss = 0
    for ii, ll in enumerate(lens):
        seq_true, seq_pred = y[ii,:int(ll)].squeeze(), y_pred[ii,:int(ll)].squeeze()
        loss += criterion(seq_pred, seq_true) / len(lens)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

    epoch_loss += loss.item()

return epoch_loss / len(train_loader)

RuntimeError: all elements of input should be between 0 and 1

In [25]:
seq_pred

tensor([nan, nan, nan,  ..., nan, nan, nan], grad_fn=<SqueezeBackward0>)

In [27]:
model

RNN(
  (rnn): RNN(3, 64, num_layers=2, batch_first=True, dropout=0.5)
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.5, inplace=False)
)